In [1]:
%load_ext autoreload
%autoreload 2

import glob
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from crc_scripts import config
from abg_python.galaxy.gal_utils import Galaxy
from abg_python.snapshot_utils import openSnapshot

from firestudio.utils.camera_utils import Camera
from firestudio.studios.star_studio import StarStudio
from firestudio.studios.gas_studio import GasStudio
matplotlib.rcParams['figure.dpi'] = 150

# First setup directory for all the plots
data_dir = '/N/scratch/cchoban/FIRE_studio/'
os.makedirs(data_dir, exist_ok=True)

Missing firefly, obtain it at http://github.com/ageller/Firefly or pip install firefly.


In [54]:
# Main directory
main_dirc = '/N/scratch/cchoban/HiZ_dust/'
# Names of simulation directories
names = np.array(['z5m11d','z5m12a','z5m12b','z5m12d',
                  'z7m12a','z7m12b','z7m12c','z9m12a',
                  'z5m12a_4acc','z5m12a_0.9sd',
                  'z5m11d_0.2sd_4acc','z5m12a_0.2sd_4acc','z5m12b_0.2sd_4acc','z5m12d_0.2sd_4acc',
                  'z7m12a_0.2sd_4acc','z7m12b_0.2sd_4acc','z7m12c_0.2sd_4acc','z9m12a_0.2sd_4acc',
                  'z5m12a_0.2sd_4acc_Tcut','z5m12a_0.2sd_4acc_FIRE3'])


short_names = np.array(['z5m11d','z5m12a','z5m12b','z5m12d',
                  'z7m12a','z7m12b','z7m12c','z9m12a',
                  'z5m12a_4acc','z5m12a_0.9sd',
                  'z5m11d_enh','z5m12a_enh','z5m12b_enh','z5m12d_enh',
                  'z7m12a_enh','z7m12b_enh','z7m12c_enh','z9m12a_enh',
                  'z5m12a_enh_Tcut','z5m12a_enh_FIRE3'])


# Some masks to get subsets of the simulations
all_mask = np.arange(len(names))
fiducial_mask = np.char.find(names, "_") < 0
z5m12a_mask = np.char.find(names, "z5m12a") >= 0
z5m12d_mask = np.char.find(names, "z5m12d") >= 0
z5m12b_mask = np.char.find(names, "z5m12b") >= 0
z5enh_mask = (np.char.find(names, "z5") >= 0) & (np.char.find(names, "0.2sd_4acc") >= 0) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))
z7enh_mask = (np.char.find(names, "z7") >= 0) & (np.char.find(names, "0.2sd_4acc") >= 0) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))
z9enh_mask = (np.char.find(names, "z9") >= 0) & (np.char.find(names, "0.2sd_4acc") >= 0) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))
z79enh_mask = ((np.char.find(names, "z9") >= 0) | (np.char.find(names, "z7") >= 0)) & (np.char.find(names, "0.2sd_4acc") >= 0) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))

z5fid_mask = (np.char.find(names, "z5") >= 0) & (np.char.find(names, "_") < 0)
z7fid_mask = (np.char.find(names, "z7") >= 0) & (np.char.find(names, "_") < 0)
z9fid_mask = (np.char.find(names, "z9") >= 0) & (np.char.find(names, "_") < 0)
z79fid_mask = ((np.char.find(names, "z9") >= 0) | (np.char.find(names, "z7") >= 0)) & (np.char.find(names, "_") < 0)

z5m12b_z7m12c_mask = ((np.char.find(names, "z5m12b") >= 0) | (np.char.find(names, "z7m12c") >= 0)) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))

z5m11d_z5m12d_mask = ((np.char.find(names, "z5m11d") >= 0) | (np.char.find(names, "z5m12d") >= 0)) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))
z7m12a_z7m12b_mask = ((np.char.find(names, "z7m12a") >= 0) | (np.char.find(names, "z7m12b") >= 0)) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))
z9m12a_mask = ((np.char.find(names, "z9m12a") >= 0)) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))

FIRE3_mask = np.char.find(names, "FIRE3") >= 0
enhanced_mask = (np.char.find(names, "0.2sd_4acc") >= 0) & ~((np.char.find(names, "0.2sd_4acc_") >= 0))

fid_and_enh_mask = (np.char.find(names, "_") < 0) | ((np.char.find(names, "0.2sd_4acc") >= 0) & ~(np.char.find(names, "0.2sd_4acc_") >= 0))



FIRE_vers = np.array([2]*(len(names)-1)+[3])


# Main sim directories and their snap and halo directories
# Note you need to specify the dtype or else you get numpy strings which annoying to deal with
main_dirs = np.array([main_dirc+name+'/' for name in names], dtype=object)
snap_dirs = np.array([main_dirc+name+'/output/' for name in names], dtype=object)
hdirs = np.array([main_dirc+name+'/halo/ahf/output/' for name in names], dtype=object)
# We only look at the main halos
history_name = 'halo_main.dat'
halo_histories = np.array([main_dirc+name+'/halo/ahf/history/'+history_name for name in names], dtype=object)

# Final snapshot number since sims are run to varying redshift
final_snaps = np.zeros(len(names),dtype=int)
for i,snap_dir in enumerate(snap_dirs):
    snap_files = os.listdir(snap_dir)
    snap_nums = []
    for snap_file in snap_files:
        if 'snapshot' in snap_file:
            if snap_file[9:12] == '000':
                snap_nums += [0]
            else:
                snap_nums += [int(snap_file[9:12].lstrip('0'))]
    final_snaps[i]=np.max(snap_nums)

print(final_snaps)
first_snaps = np.full(len(names),66)

# Set what sims you want to make movies for
mask = z9enh_mask
# How to account for dust ('none'= no dust, 'live' = use live dust results, 'fixed'=set D/Z everywhere)
dust_opt='live'

# Set the image size and resolutionabs
# Sets the image halfwidth size scaled with a fraction of the virial radius
image_rvirfrac = 1
image_scale_len = 5 # kpc
pixel_res = 0.02
#pixels = int(2*image_halfwidth/pixel_res)
fontsize=75
# Can either used fixed stellar hsml or compute them on the fly
fixed_star_hsml=None

# Useful to set the dynamic light range since the default makes the entire halo glow
set_maxden=None
set_dynrange=500

# Interval between snapshots projections
first_snap = 10
snap_interval=1

[67 67 66 67 32 35 41 16 67 67 67 67 64 67 33 33 41 18 67 41]


# Gas Projection

## First get projection of main galaxy from last snapshot. This will determine the camera orientation.

In [55]:
# Set the image size and resolutionabs
# Sets the image halfwidth size scaled with a fraction of the virial radius
image_rvirfrac = 1
image_scale_len = 5 # kpc
pixel_res = 0.02
#pixels = int(2*image_halfwidth/pixel_res)
fontsize=75

# Set range of projection values
weight_lims=[-1,4]  # log10

# Interval between snapshots projections
first_snap = 10
snap_interval=1

In [56]:
# First get face-on axis to force camera position for all projections
force_phi_TBs=[];force_theta_TBs=[]
image_halfwidths = []
for i,name in enumerate(names[mask]):
    studio_datadir = os.path.join(data_dir,'firestudio/'+name+'_proj_movie')
    snapnum=final_snaps[mask][i]
    snap_dir = snap_dirs[mask][i]
    halo_history = halo_histories[mask][i]
    hdir = hdirs[mask][i]
    FIRE_ver=FIRE_vers[mask][i]


    history_dirc = main_dirc+name+'/halo/ahf/history/'
    galaxy = Galaxy(
            name,
            snapnum,
            suite_name = 'dust',
            snapdir=snap_dir,
            datadir=data_dir+'subsnaps',
            halo_path=history_dirc,
            halo_fname='halo_main.dat',
            loud=True
            )
    
    print(history_dirc)
    print(galaxy.scom,galaxy.rvir)
    # It seems the last snapshot's redshift is slightly off from the expected redshift. 
    # So the redshift listed in the halo file is wrong and leads to an incorrect conversion. 
    # Set it manually to fix this....
    hubble = galaxy.header['HubbleParam']
    ascale = galaxy.header['Time']
    Xc, Yc, Zc, Rvir = ascale/hubble*np.loadtxt(halo_history, skiprows=1,usecols=(7,8,9,13,), unpack=True)
    nums = np.loadtxt(halo_history, skiprows=1,usecols=(0,), unpack=True)
    idx = nums==snapnum
    Xc = Xc[idx][0];Yc = Yc[idx][0];Zc = Zc[idx][0];Rvir = Rvir[idx][0];
    galaxy.rvir=Rvir
    galaxy.scom=np.array([Xc, Yc, Zc])
    print(galaxy.rvir,galaxy.scom)

    # Keep all of the snapshot data for now
    galaxy.extractMainHalo(free_mem=True,use_metadata=True,orient_component='star')

    force_theta_TBs += [galaxy.theta_TB]
    force_phi_TBs += [galaxy.phi_TB]

    # Makes projection face on with the disk
    image_halfwidth = galaxy.rvir*image_rvirfrac
    pixels = int(2*image_halfwidth/pixel_res)
    camera_pos = np.array([0,0,image_halfwidth])
    print('image_halfwidth:',image_halfwidth)
    
    ## initialize the GasStudio instance
    my_gasStudio = GasStudio(
        studio_datadir,
        galaxy.snapnum,
        studio_datadir,
        gas_snapdict=galaxy.sub_snap, ## pass in snapshot dictionary
        star_snapdict=galaxy.sub_star_snap, ## pass in snapshot dictionary
        frame_half_width=image_halfwidth,
        pixels=pixels,
        scale_line_length=image_scale_len,
        figure_label='z = %.3f'%galaxy.current_redshift,
        fontsize=fontsize,
        camera_pos = camera_pos,
        savefig=name+'_mass_proj_'+str(image_rvirfrac)+'Rvir_'+str(galaxy.snapnum),
        )

    # my_gasStudio.set_ImageParams(
    #     use_colorbar=True,
    #     cbar_label='$\Sigma_g$ (Msun pc$^{-2}$)',
    #     scale_line_length=image_scale_len,
    #     cbar_logspace=True,
    #     theta=0,
    #     aspect_ratio=1)

    my_gasStudio.render(
        weight_name='Masses',
        quantity_name='Temperature',
        quantity_adjustment_function=np.log10,    
        min_weight=weight_lims[0],
        max_weight=weight_lims[1],
        weight_adjustment_function=lambda x: np.log10(x/my_gasStudio.Acell) + 10 - 6, ## msun/pc^2
        cmap='afmhot')
    plt.close()

    image_halfwidths+=[image_halfwidth]

    del(galaxy)

Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/metadata/meta_Galaxy_018.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/halo/ahf/history/
[7493.29101005 7517.55200116 7681.05559552] 17.466623122145087
17.466618933823526 [7493.28921324 7517.55019853 7681.05375368]
Loading star particles of z9m12a_0.2sd_4acc at 18 at /N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/output
compute_rstar_half(z9m12a_0.2sd_4acc at 18,){'within_radius': 8.733309466911763} 1.07 s elapsed
Using the saved sub-snapshots for z9m12a_0.2sd_4acc at 18
Successfully loaded a pre-extracted subsnap
Reorienting...
Done.
extract_halo_inner(z9m12a_0.2sd_4acc at 18,){'orient_component': 'star'} 10.29 s elapsed
Snapshot memory free
image_halfwidth: 17.466618933823526
setting use_colorbar to default value of: False
setting cbar_label to default value of: 
setting cbar_logspace to default

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/gas_studio.py:766: RuntimeWarning: invalid value encountered in divide
  weightWeightedQuantityMap = weightWeightedQuantityMap/weightMap
/tmp/ipykernel_2784969/270816406.py:82: RuntimeWarning: divide by zero encountered in log10
  weight_adjustment_function=lambda x: np.log10(x/my_gasStudio.Acell) + 10 - 6, ## msun/pc^2


min_Masses =  -1
max_Masses =  4
Image range (Masses):  -inf 3.6330023
Image range (8bit):  0.0 236.28311


## Now make projection for specfied snapshots

In [57]:
# First get face-on axis to force camera position for all projections
for i,name in enumerate(names[mask]):
    studio_datadir = os.path.join(data_dir,'firestudio/'+name+'_proj_movie')
    snapnum=final_snaps[mask][i]
    snap_dir = snap_dirs[mask][i]
    halo_history = halo_histories[mask][i]
    hdir = hdirs[mask][i]
    FIRE_ver=FIRE_vers[mask][i]
    last_snap=snapnum=final_snaps[mask][i]

    history_dirc = main_dirc+name+'/halo/ahf/history/'
    first_snap=10
    for j, snapnum in enumerate(np.arange(first_snap,last_snap,snap_interval)):
        galaxy = Galaxy(
            name,
            snapnum,
            suite_name = 'dust',
            snapdir=snap_dir,
            datadir=data_dir+'subsnaps',
            halo_path=history_dirc,
            halo_fname='halo_main.dat',
            loud=True
            )
        
        print(history_dirc)
        print(galaxy.scom,galaxy.rvir)
        # It seems the last snapshot's redshift is slightly off from the expected redshift. 
        # So the redshift listed in the halo file is wrong and leads to an incorrect conversion. 
        # Set it manually to fix this....
        hubble = galaxy.header['HubbleParam']
        ascale = galaxy.header['Time']
        Xc, Yc, Zc, Rvir = ascale/hubble*np.loadtxt(halo_history, skiprows=1,usecols=(7,8,9,13,), unpack=True)
        nums = np.loadtxt(halo_history, skiprows=1,usecols=(0,), unpack=True)
        idx = nums==snapnum
        Xc = Xc[idx][0];Yc = Yc[idx][0];Zc = Zc[idx][0];Rvir = Rvir[idx][0];
        galaxy.rvir=Rvir
        galaxy.scom=np.array([Xc, Yc, Zc])
        print(galaxy.rvir,galaxy.scom)
    
        # Keep all of the snapshot data for now
        galaxy.extractMainHalo(free_mem=True,use_metadata=True,orient_component='star',
                                   force_phi_TB=force_phi_TBs[i],force_theta_TB=force_theta_TBs[i])
    
        # Makes projection face on with the disk
        image_halfwidth = image_halfwidths[i]
        pixels = int(2*image_halfwidth/pixel_res)
        camera_pos = np.array([0,0,image_halfwidth])
        print('image_halfwidth:',image_halfwidth)
    
        ## initialize the GasStudio instance
        my_gasStudio = GasStudio(
            studio_datadir,
            galaxy.snapnum,
            studio_datadir,
            gas_snapdict=galaxy.sub_snap, ## pass in snapshot dictionary
            star_snapdict=galaxy.sub_star_snap, ## pass in snapshot dictionary
            frame_half_width=image_halfwidth,
            pixels=pixels,
            scale_line_length=image_scale_len,
            figure_label='z = %.3f'%galaxy.current_redshift,
            fontsize=fontsize,
            camera_pos = camera_pos,
            savefig=name+'_mass_proj_'+str(image_rvirfrac)+'Rvir_'+str(galaxy.snapnum),
            )
    
        # my_gasStudio.set_ImageParams(
        #     use_colorbar=True,
        #     cbar_label='$\Sigma_g$ (Msun pc$^{-2}$)',
        #     scale_line_length=image_scale_len,
        #     cbar_logspace=True,
        #     theta=0,
        #     aspect_ratio=1)
    
        my_gasStudio.render(
            weight_name='Masses',
            quantity_name='Temperature',
            quantity_adjustment_function=np.log10,    
            min_weight=weight_lims[0],
            max_weight=weight_lims[1],
            weight_adjustment_function=lambda x: np.log10(x/my_gasStudio.Acell) + 10 - 6, ## msun/pc^2
            cmap='afmhot')
        plt.close()
    
        del(galaxy)

Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/metadata/meta_Galaxy_010.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/halo/ahf/history/
[6414.01532968 6439.64664353 6571.44904618] 9.184185861176879
9.184190175 [6414.01834235 6439.64966824 6571.45213279]
Loading star particles of z9m12a_0.2sd_4acc at 10 at /N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/output
compute_rstar_half(z9m12a_0.2sd_4acc at 10,){'within_radius': 4.5920950875} 0.26 s elapsed
Using the saved sub-snapshots for z9m12a_0.2sd_4acc at 10
Failed to open saved sub-snapshots FileNotFoundError(2, "Unable to open file (unable to open file: name = '/N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/subsnaps/snapshot_010.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)")
Loading gas particles of z9m12a_0.2sd_4acc at 10 at /N/scratch/cchoba

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/gas_studio.py:766: RuntimeWarning: invalid value encountered in divide
  weightWeightedQuantityMap = weightWeightedQuantityMap/weightMap
/tmp/ipykernel_2784969/2691542782.py:80: RuntimeWarning: divide by zero encountered in log10
  weight_adjustment_function=lambda x: np.log10(x/my_gasStudio.Acell) + 10 - 6, ## msun/pc^2


min_Masses =  -1
max_Masses =  4
Image range (Masses):  -inf 3.2626553
Image range (8bit):  0.0 217.39543
Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/metadata/meta_Galaxy_011.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/halo/ahf/history/
[6548.65058679 6574.59477135 6710.18133591] 9.933238664583065
9.933232749999998 [6548.6466875  6574.59085662 6710.17734044]
Loading star particles of z9m12a_0.2sd_4acc at 11 at /N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/output
compute_rstar_half(z9m12a_0.2sd_4acc at 11,){'within_radius': 4.966616374999999} 0.24 s elapsed
Using the saved sub-snapshots for z9m12a_0.2sd_4acc at 11
Failed to open saved sub-snapshots FileNotFoundError(2, "Unable to open file (unable to open file: name = '/N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/subsnaps/snapshot_011.hdf5', errno = 2, error message = 'No suc

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/gas_studio.py:766: RuntimeWarning: invalid value encountered in divide
  weightWeightedQuantityMap = weightWeightedQuantityMap/weightMap
/tmp/ipykernel_2784969/2691542782.py:80: RuntimeWarning: divide by zero encountered in log10
  weight_adjustment_function=lambda x: np.log10(x/my_gasStudio.Acell) + 10 - 6, ## msun/pc^2


Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/metadata/meta_Galaxy_012.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/halo/ahf/history/
[6683.16598476 6709.41375567 6849.28674538] 11.28999514453077
11.28999146470588 [6683.16380647 6709.41156882 6849.28451294]
Loading star particles of z9m12a_0.2sd_4acc at 12 at /N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/output
compute_rstar_half(z9m12a_0.2sd_4acc at 12,){'within_radius': 5.64499573235294} 0.28 s elapsed
Using the saved sub-snapshots for z9m12a_0.2sd_4acc at 12
Failed to open saved sub-snapshots FileNotFoundError(2, "Unable to open file (unable to open file: name = '/N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/subsnaps/snapshot_012.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)")
Loading gas particles of z9m12a_0.2sd_4acc at 12 at /N/scra

...

Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/metadata/meta_Galaxy_016.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/halo/ahf/history/
[7223.1261936  7249.37582118 7403.7634412 ] 15.542018831338517
15.542026882352939 [7223.12993529 7249.37957647 7403.76727647]
Loading star particles of z9m12a_0.2sd_4acc at 16 at /N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/output
compute_rstar_half(z9m12a_0.2sd_4acc at 16,){'within_radius': 7.7710134411764695} 0.69 s elapsed
Using the saved sub-snapshots for z9m12a_0.2sd_4acc at 16
Failed to open saved sub-snapshots FileNotFoundError(2, "Unable to open file (unable to open file: name = '/N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/subsnaps/snapshot_016.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)")
Loading gas particles of z9m12a_0.2sd_4acc at 16 at /N/

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/gas_studio.py:766: RuntimeWarning: invalid value encountered in divide
  weightWeightedQuantityMap = weightWeightedQuantityMap/weightMap
/tmp/ipykernel_2784969/2691542782.py:80: RuntimeWarning: divide by zero encountered in log10
  weight_adjustment_function=lambda x: np.log10(x/my_gasStudio.Acell) + 10 - 6, ## msun/pc^2


Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/metadata/meta_Galaxy_017.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/halo/ahf/history/
[7357.8860571  7383.23142165 7542.6537647 ] 16.586006564692735
16.58599896470588 [7357.88268559 7383.22803853 7542.65030853]
Loading star particles of z9m12a_0.2sd_4acc at 17 at /N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/output
compute_rstar_half(z9m12a_0.2sd_4acc at 17,){'within_radius': 8.29299948235294} 0.80 s elapsed
Using the saved sub-snapshots for z9m12a_0.2sd_4acc at 17
Failed to open saved sub-snapshots FileNotFoundError(2, "Unable to open file (unable to open file: name = '/N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/subsnaps/snapshot_017.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)")
Loading gas particles of z9m12a_0.2sd_4acc at 17 at /N/scr

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/gas_studio.py:766: RuntimeWarning: invalid value encountered in divide
  weightWeightedQuantityMap = weightWeightedQuantityMap/weightMap
/tmp/ipykernel_2784969/2691542782.py:80: RuntimeWarning: divide by zero encountered in log10
  weight_adjustment_function=lambda x: np.log10(x/my_gasStudio.Acell) + 10 - 6, ## msun/pc^2


min_Masses =  -1
max_Masses =  4
Image range (Masses):  -inf 3.8619814
Image range (8bit):  0.0 247.96104


# Star Projection

## First make a nice image of the last snap.

In [9]:
# Set the image size and resolutionabs
# Sets the image halfwidth size scaled with a fraction of the virial radius
image_rvirfrac = 1
image_scale_len = 2 # kpc
pixel_res = 0.02
#pixels = int(2*image_halfwidth/pixel_res)
fontsize=75
# Can either used fixed stellar hsml or compute them on the fly
fixed_star_hsml=None

# Useful to set the dynamic light range since the default makes the entire halo glow
set_maxden=None
set_dynrange=700

# Interval between snapshots projections
first_snap = 10
snap_interval=2

In [12]:
# First get face-on axis to force camera position for all projections
maxdens=[]; dynranges=[];
force_phi_TBs=[];force_theta_TBs=[]
for i,name in enumerate(names[mask]):
    studio_datadir = os.path.join(data_dir,'firestudio/'+name+'_hubble_movie')
    snapnum=final_snaps[mask][i]
    snap_dir = snap_dirs[mask][i]
    halo_history = halo_histories[mask][i]
    hdir = hdirs[mask][i]
    FIRE_ver=FIRE_vers[mask][i]

    history_dirc = main_dirc+name+'/halo/ahf/history/'
    galaxy = Galaxy(
        name,
        snapnum,
        suite_name = 'dust',
        snapdir=snap_dir,
        datadir=data_dir+'subsnaps',
        halo_path=history_dirc,
        halo_fname='halo_main.dat',
        loud=True
        )
    print(history_dirc)
    print(galaxy.scom,galaxy.rvir)
    # It seems the last snapshot's redshift is slightly off from the expected redshift. 
    # So the redshift listed in the halo file is wrong and leads to an incorrect conversion. 
    # Set it manually to fix this....
    hubble = galaxy.header['HubbleParam']
    ascale = galaxy.header['Time']
    Xc, Yc, Zc, Rvir = ascale/hubble*np.loadtxt(halo_history, skiprows=1,usecols=(7,8,9,13,), unpack=True)
    nums = np.loadtxt(halo_history, skiprows=1,usecols=(0,), unpack=True)
    idx = nums==snapnum
    Xc = Xc[idx][0];Yc = Yc[idx][0];Zc = Zc[idx][0];Rvir = Rvir[idx][0];
    galaxy.rvir=Rvir
    galaxy.scom=np.array([Xc, Yc, Zc])
    print(galaxy.rvir,galaxy.scom)

    # Keep all of the snapshot data for now
    galaxy.extractMainHalo(free_mem=True,use_metadata=True,orient_component='star',force_theta_TB=None,
            force_phi_TB=None,use_saved_subsnapshots=True)
    force_theta_TBs += [galaxy.theta_TB]
    force_phi_TBs += [galaxy.phi_TB]
    if dust_opt == 'none':
        live_dust=False; no_dust=True;
    elif dust_opt == 'live':
        live_dust=True; no_dust=False;
    else:
        live_dust=False; no_dust=False;

    # Makes projection face on with the disk
    image_halfwidth = galaxy.rvir*image_rvirfrac
    pixels = int(2*image_halfwidth/pixel_res)
    camera_pos = np.array([0,0,image_halfwidth])
    print('image_halfwidth:',image_halfwidth)

    starStudio = StarStudio(
        studio_datadir,
        galaxy.snapnum,
        galaxy.datadir_name,
        gas_snapdict=galaxy.sub_snap,
        star_snapdict=galaxy.sub_star_snap,
        live_dust=live_dust,
        no_dust=no_dust,
        figure_label=name.split('_')[0],
        savefig=name+'_'+dust_opt+'dust_'+str(image_rvirfrac)+'Rvir_'+str(galaxy.snapnum),
        frame_half_width=image_halfwidth,
        pixels=pixels,
        scale_line_length=image_scale_len,
        fontsize=fontsize,
        maxden=set_maxden,
        dynrange=set_dynrange)


    ax,final_image = starStudio.render(
        plt.gca(),
        quick=False,
        use_metadata=True,
        fixed_star_hsml=fixed_star_hsml)
    plt.gcf().set_dpi(120)
    plt.close()

    print('maxden:', starStudio.maxden,'dynrange:',starStudio.dynrange)

    #maxden: 3053007921.712722 dynrange: 383.93820590700943
    
    maxdens+=[starStudio.maxden]
    dynranges+=[starStudio.dynrange]

Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z5m11d_0.2sd_4acc/metadata/meta_Galaxy_067.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z5m11d_0.2sd_4acc/halo/ahf/history/
[3758.57843137 3720.34313725 3585.29411765] 26.02941176470588
25.925294117647038 [3743.54411765 3705.46176471 3570.95294118]
Loading star particles of z5m11d_0.2sd_4acc at 67 at /N/scratch/cchoban/HiZ_dust/z5m11d_0.2sd_4acc/output
compute_rstar_half(z5m11d_0.2sd_4acc at 67,){'within_radius': 12.962647058823519} 0.11 s elapsed
Using the saved sub-snapshots for z5m11d_0.2sd_4acc at 67
Successfully loaded a pre-extracted subsnap
Reorienting...
Done.
extract_halo_inner(z5m11d_0.2sd_4acc at 67,){'orient_component': 'star', 'force_theta_TB': None, 'force_phi_TB': None, 'use_saved_subsnapshots': True} 2.48 s elapsed
Snapshot memory free
image_halfwidth: 25.925294117647038
setting live_dust to user value of: True
set

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/star_studio.py:595: RuntimeWarning: divide by zero encountered in log10
  rats = np.log10(L_sfc_densities.flatten())


setting maxden to best guess 5.6e+07
setting maxden to user value of: 55612998.965453975
maxden ==  55612998.965453975 dynrange ==  700 minden ==  79447.14137921996


/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:71: RuntimeWarning: divide by zero encountered in log
  r = np.log(r/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:72: RuntimeWarning: divide by zero encountered in log
  g = np.log(g/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:73: RuntimeWarning: divide by zero encountered in log
  b = np.log(b/minnorm) / np.log(maxnorm/minnorm)


maxden: 55612998.965453975 dynrange: 700
Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z5m12a_0.2sd_4acc/metadata/meta_Galaxy_067.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z5m12a_0.2sd_4acc/halo/ahf/history/
[3400.24509804 3548.52941176 3559.06862745] 39.28921568627451
39.132058823529384 [3386.64411765 3534.33529412 3544.83235294]
Loading star particles of z5m12a_0.2sd_4acc at 67 at /N/scratch/cchoban/HiZ_dust/z5m12a_0.2sd_4acc/output
compute_rstar_half(z5m12a_0.2sd_4acc at 67,){'within_radius': 19.566029411764692} 0.83 s elapsed
Using the saved sub-snapshots for z5m12a_0.2sd_4acc at 67
Successfully loaded a pre-extracted subsnap
Reorienting...
Done.
extract_halo_inner(z5m12a_0.2sd_4acc at 67,){'orient_component': 'star', 'force_theta_TB': None, 'force_phi_TB': None, 'use_saved_subsnapshots': True} 13.73 s elapsed
Snapshot memory free
image_halfwidth: 39.132058823529384
se

...

setting maxden to best guess 8e+08
setting maxden to user value of: 798295177.2197819
maxden ==  798295177.2197819 dynrange ==  700 minden ==  1140421.6817425455


/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:71: RuntimeWarning: divide by zero encountered in log
  r = np.log(r/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:72: RuntimeWarning: divide by zero encountered in log
  g = np.log(g/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:73: RuntimeWarning: divide by zero encountered in log
  b = np.log(b/minnorm) / np.log(maxnorm/minnorm)


maxden: 798295177.2197819 dynrange: 700
Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z9m12a_0.2sd_4acc/metadata/meta_Galaxy_018.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/halo/ahf/history/
[7493.29101005 7517.55200116 7681.05559552] 17.466623122145087
17.466618933823526 [7493.28921324 7517.55019853 7681.05375368]
Loading star particles of z9m12a_0.2sd_4acc at 18 at /N/scratch/cchoban/HiZ_dust/z9m12a_0.2sd_4acc/output
compute_rstar_half(z9m12a_0.2sd_4acc at 18,){'within_radius': 8.733309466911763} 1.12 s elapsed
Using the saved sub-snapshots for z9m12a_0.2sd_4acc at 18
Successfully loaded a pre-extracted subsnap
Reorienting...
Done.
extract_halo_inner(z9m12a_0.2sd_4acc at 18,){'orient_component': 'star', 'force_theta_TB': None, 'force_phi_TB': None, 'use_saved_subsnapshots': True} 9.84 s elapsed
Snapshot memory free
image_halfwidth: 17.466618933823526
sett

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/star_studio.py:595: RuntimeWarning: divide by zero encountered in log10
  rats = np.log10(L_sfc_densities.flatten())
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:71: RuntimeWarning: divide by zero encountered in log
  r = np.log(r/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:72: RuntimeWarning: divide by zero encountered in log
  g = np.log(g/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:73: RuntimeWarning: divide by zero encountered in log
  b = np.log(b/minnorm) / np.log(maxnorm/minnorm)


setting maxden to best guess 2.7e+09
setting maxden to user value of: 2704576949.662627
maxden ==  2704576949.662627 dynrange ==  700 minden ==  3863681.356660896
maxden: 2704576949.662627 dynrange: 700


## Movie Making: Get a wide image of the last snap to set the orientation, brightness, and image width

In [11]:
# Set the image size and resolutionabs
# Sets the image halfwidth size scaled with a fraction of the virial radius
image_rvirfrac = 1
image_scale_len = 2 # kpc
pixel_res = 0.02
#pixels = int(2*image_halfwidth/pixel_res)
fontsize=75
# Can either used fixed stellar hsml or compute them on the fly
fixed_star_hsml=None

# Useful to set the dynamic light range since the default makes the entire halo glow
set_maxden=None
set_dynrange=700

# Interval between snapshots projections
first_snap = 10
snap_interval=2

In [23]:
# First get face-on axis to force camera position for all projections
maxdens=[]; dynranges=[];
force_phi_TBs=[];force_theta_TBs=[]
image_halfwidths = []
for i,name in enumerate(names[mask]):
    studio_datadir = os.path.join(data_dir,'firestudio/'+name+'_hubble_movie')
    snapnum=final_snaps[mask][i]
    snap_dir = snap_dirs[mask][i]
    halo_history = halo_histories[mask][i]
    hdir = hdirs[mask][i]
    FIRE_ver=FIRE_vers[mask][i]

    history_dirc = main_dirc+name+'/halo/ahf/history/'
    galaxy = Galaxy(
        name,
        snapnum,
        suite_name = 'dust',
        snapdir=snap_dir,
        datadir=data_dir+'subsnaps',
        halo_path=history_dirc,
        halo_fname='halo_main.dat',
        loud=True
        )
    print(history_dirc)
    print(galaxy.scom,galaxy.rvir)
    # It seems the last snapshot's redshift is slightly off from the expected redshift. 
    # So the redshift listed in the halo file is wrong and leads to an incorrect conversion. 
    # Set it manually to fix this....
    hubble = galaxy.header['HubbleParam']
    ascale = galaxy.header['Time']
    Xc, Yc, Zc, Rvir = ascale/hubble*np.loadtxt(halo_history, skiprows=1,usecols=(7,8,9,13,), unpack=True)
    nums = np.loadtxt(halo_history, skiprows=1,usecols=(0,), unpack=True)
    idx = nums==snapnum
    Xc = Xc[idx][0];Yc = Yc[idx][0];Zc = Zc[idx][0];Rvir = Rvir[idx][0];
    galaxy.rvir=Rvir
    galaxy.scom=np.array([Xc, Yc, Zc])
    print(galaxy.rvir,galaxy.scom)

    # Keep all of the snapshot data for now
    galaxy.extractMainHalo(free_mem=True,use_metadata=True,orient_component='star',force_theta_TB=None,
            force_phi_TB=None,use_saved_subsnapshots=True)
    force_theta_TBs += [galaxy.theta_TB]
    force_phi_TBs += [galaxy.phi_TB]
    if dust_opt == 'none':
        live_dust=False; no_dust=True;
    elif dust_opt == 'live':
        live_dust=True; no_dust=False;
    else:
        live_dust=False; no_dust=False;

    # Makes projection face on with the disk
    image_halfwidth = galaxy.rvir*image_rvirfrac
    image_halfwidths+=[image_halfwidth]
    pixels = int(2*image_halfwidth/pixel_res)
    camera_pos = np.array([0,0,image_halfwidth])
    print('image_halfwidth:',image_halfwidth)

    starStudio = StarStudio(
    studio_datadir,
    galaxy.snapnum,
    galaxy.datadir_name,
    gas_snapdict=galaxy.sub_snap,
    star_snapdict=galaxy.sub_star_snap,
    live_dust=live_dust,
    no_dust=no_dust,
    figure_label=name.split('_')[0],
    savefig=name+'movie_'+dust_opt+'dust_'+str(image_rvirfrac)+'Rvir_'+str(galaxy.snapnum),
    frame_half_width=image_halfwidth,
    pixels=pixels,
    scale_line_length=image_scale_len,
    fontsize=fontsize,
    maxden=set_maxden,
    dynrange=set_dynrange)


    ax,final_image = starStudio.render(
        plt.gca(),
        quick=False,
        use_metadata=True,
        fixed_star_hsml=fixed_star_hsml)
    plt.gcf().set_dpi(120)
    plt.close()

    print('maxden:', starStudio.maxden,'dynrange:',starStudio.dynrange)

    #maxden: 3053007921.712722 dynrange: 383.93820590700943
    
    maxdens+=[starStudio.maxden]
    dynranges+=[starStudio.dynrange]

Couldn't find a metadata file... for
 Metadata object at /N/scratch/cchoban/FIRE_studio/subsnaps/z5m12a_0.2sd_4acc/metadata/meta_Galaxy_067.hdf5
We don't use ahf, only rockstar. join us. 	but we'll allow it this time.
/N/scratch/cchoban/HiZ_dust/z5m12a_0.2sd_4acc/halo/ahf/history/
[3400.24509804 3548.52941176 3559.06862745] 39.28921568627451
39.132058823529384 [3386.64411765 3534.33529412 3544.83235294]
Loading star particles of z5m12a_0.2sd_4acc at 67 at /N/scratch/cchoban/HiZ_dust/z5m12a_0.2sd_4acc/output
compute_rstar_half(z5m12a_0.2sd_4acc at 67,){'within_radius': 19.566029411764692} 2.50 s elapsed
Using the saved sub-snapshots for z5m12a_0.2sd_4acc at 67
Successfully loaded a pre-extracted subsnap
Reorienting...
Done.
extract_halo_inner(z5m12a_0.2sd_4acc at 67,){'orient_component': 'star', 'force_theta_TB': None, 'force_phi_TB': None, 'use_saved_subsnapshots': True} 19.88 s elapsed
Snapshot memory free
image_halfwidth: 39.132058823529384
setting live_dust to user value of: True
se

/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/colors_sps/read_band_lums_from_tables.py:171: RuntimeWarning: divide by zero encountered in log10
  np.log10(age_in_Gyr)+9.0,


Calculating L/M in Sloan g (BAND_ID=2,l=4686 A)
Calculating L/M in Sloan r (BAND_ID=3,l=6165 A)
4708594 many gas particles in volume
Using live dust fields for Hubble image!


/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/star_studio.py:445: RuntimeWarning: invalid value encountered in divide
  sc_ratios = np.where(carb_frac>0, sil_frac/carb_frac, 1E4)


Si/C ratio is : 14.293272
Will be using LMC-type dust for extinction...
Si/C ratio is : 14.293272
Will be using LMC-type dust for extinction...
Si/C ratio is : 14.293272
Will be using LMC-type dust for extinction...
Projecting with attenuation...
total lum before attenuation in each band (Lsun/1e10): [12.83896753  5.63652362  2.93616135]
opacity in each band: [1580.7683296532966, 1194.471743727964, 852.2784055757957]
total attentuation gas mass: 0.09176795
compute_mockHubbleImage(StarStudio instance,){'fixed_star_hsml': None} 489.16 s elapsed


...

Calculating L/M in Sloan r (BAND_ID=3,l=6165 A)
3665253 many gas particles in volume
Using live dust fields for Hubble image!
Si/C ratio is : 13.985903
Will be using LMC-type dust for extinction...
Si/C ratio is : 13.985903
Will be using LMC-type dust for extinction...
Si/C ratio is : 13.985903
Will be using LMC-type dust for extinction...


/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/star_studio.py:445: RuntimeWarning: divide by zero encountered in divide
  sc_ratios = np.where(carb_frac>0, sil_frac/carb_frac, 1E4)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/star_studio.py:445: RuntimeWarning: invalid value encountered in divide
  sc_ratios = np.where(carb_frac>0, sil_frac/carb_frac, 1E4)


Projecting with attenuation...
total lum before attenuation in each band (Lsun/1e10): [68.11404078 21.08477201 10.39682754]
opacity in each band: [1580.7683296532966, 1194.471743727964, 852.2784055757957]
total attentuation gas mass: 0.06862198
compute_mockHubbleImage(StarStudio instance,){'fixed_star_hsml': None} 150.29 s elapsed


/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/studios/star_studio.py:595: RuntimeWarning: divide by zero encountered in log10
  rats = np.log10(L_sfc_densities.flatten())
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:71: RuntimeWarning: divide by zero encountered in log
  r = np.log(r/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:72: RuntimeWarning: divide by zero encountered in log
  g = np.log(g/minnorm) / np.log(maxnorm/minnorm)
/geode2/home/u020/cchoban/BigRed200/codes/FIRE_studio/src/firestudio/utils/stellar_utils/make_threeband_image.py:73: RuntimeWarning: divide by zero encountered in log
  b = np.log(b/minnorm) / np.log(maxnorm/minnorm)


setting maxden to best guess 2.7e+09
setting maxden to user value of: 2704576949.662627
maxden ==  2704576949.662627 dynrange ==  700 minden ==  3863681.356660896
maxden: 2704576949.662627 dynrange: 700


## Make projections for all snapshots

In [ ]:
for i,name in enumerate(names[mask]):
    studio_datadir = os.path.join(data_dir,'firestudio/'+name+'_hubble_movie')
    snap_dir = snap_dirs[mask][i]
    halo_history = halo_histories[mask][i]
    hdir = hdirs[mask][i]
    FIRE_ver=FIRE_vers[mask][i]
    history_dirc = main_dirc+name+'/halo/ahf/history/'
    last_snap=snapnum=final_snaps[mask][i]

    for j, snapnum in enumerate(np.arange(first_snap,last_snap,snap_interval)):

        galaxy = Galaxy(
            name,
            snapnum,
            suite_name = 'dust',
            snapdir=snap_dir,
            datadir=data_dir+'subsnaps',
            halo_path=history_dirc,
            halo_fname='halo_main.dat',
            loud=True
            )
        print(history_dirc)
        print(galaxy.scom,galaxy.rvir)
        # It seems the last snapshot's redshift is slightly off from the expected redshift. 
        # So the redshift listed in the halo file is wrong and leads to an incorrect conversion. 
        # Set it manually to fix this....
        hubble = galaxy.header['HubbleParam']
        ascale = galaxy.header['Time']
        Xc, Yc, Zc, Rvir = ascale/hubble*np.loadtxt(halo_history, skiprows=1,usecols=(7,8,9,13,), unpack=True)
        nums = np.loadtxt(halo_history, skiprows=1,usecols=(0,), unpack=True)
        idx = nums==snapnum
        Xc = Xc[idx][0];Yc = Yc[idx][0];Zc = Zc[idx][0];Rvir = Rvir[idx][0];
        galaxy.rvir=Rvir
        galaxy.scom=np.array([Xc, Yc, Zc])
        print(galaxy.rvir,galaxy.scom)

        # Keep all of the snapshot data for now
        force_theta_TB = force_theta_TBs[i];force_phi_TB=force_phi_TBs[i]
        galaxy.extractMainHalo(free_mem=True,use_metadata=True,orient_component='star',
                               force_phi_TB=force_phi_TB,force_theta_TB=force_theta_TB)


        # Makes projection face on with the disk
        image_halfwidth = image_halfwidths[i]
        pixels = int(2*image_halfwidth/pixel_res)
        print('image_halfwidth:',image_halfwidth)

        # Makes the projection edge on with the disk
        camera_pos = np.array([image_halfwidth,0,0])
        # Makes projection face on with the disk
        camera_pos = np.array([0,0,image_halfwidth])

        if dust_opt == 'none':
            live_dust=False; no_dust=True;
        elif dust_opt == 'live':
            live_dust=True; no_dust=False;
        else:
            live_dust=False; no_dust=False;
        
        starStudio = StarStudio(
        studio_datadir,
        galaxy.snapnum,
        galaxy.datadir_name,
        gas_snapdict=galaxy.sub_snap,
        star_snapdict=galaxy.sub_star_snap,
        live_dust=live_dust,
        no_dust=no_dust,
        figure_label='z = %.3f'%galaxy.current_redshift,
        savefig=name+'movie_'+dust_opt+'dust_'+str(image_rvirfrac)+'Rvir_'+str(galaxy.snapnum),
        frame_half_width=image_halfwidth,
        pixels=pixels,
        scale_line_length=image_scale_len,
        fontsize=fontsize,
        maxden=maxdens[i],
        dynrange=dynranges[i]
        )


        ax,final_image = starStudio.render(
            plt.gca(),
            quick=False,
            use_metadata=True,
            fixed_star_hsml=fixed_star_hsml)
        plt.gcf().set_dpi(120)
        plt.close()
    
        print('maxden:', starStudio.maxden,'dynrange:',starStudio.dynrange)

        del(galaxy)